In [1]:
!mkdir -p /content/IPCV
!mkdir -p /content/lib
!git clone https://github.com/divamgupta/image-segmentation-keras.git /content/IPCV
!git clone https://github.com/ilmissigno/IPCV.git /content/lib

Cloning into '/content/IPCV'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 742 (delta 3), reused 0 (delta 0), pack-reused 731
Receiving objects: 100% (742/742), 3.18 MiB | 19.25 MiB/s, done.
Resolving deltas: 100% (441/441), done.
Cloning into '/content/lib'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 111 (delta 27), reused 85 (delta 7), pack-reused 0
Receiving objects: 100% (111/111), 3.90 MiB | 17.07 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [0]:
#Cancellazione delle cartelle precedentemente create
import os, shutil
folder = 'semantic_drone_dataset/'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [2]:
#Caricamento delle api di kaggle -- N.B Scaricare da sopra a kaggle
#in my account , create my api key per andare a scaricare il file kaggle.json
from google.colab import files
!pip install -q kaggle
uploaded = files.upload() #Caricarlo qui selezionandolo da file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
#Caricamento del dataset da kaggle
!kaggle datasets download -d bulentsiyah/semantic-drone-dataset
#Unzip del dataset
!unzip semantic-drone-dataset.zip -d .

Saving kaggle.json to kaggle.json
kaggle.json
100% 3.83G/3.83G [01:14<00:00, 20.3MB/s]
100% 3.83G/3.83G [01:14<00:00, 55.2MB/s]
Archive:  semantic-drone-dataset.zip
  inflating: ./semantic_drone_dataset/label_images_semantic/000.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/001.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/002.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/003.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/004.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/005.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/006.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/008.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/011.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/013.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/014.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/015.p

In [3]:
!rm -rf semantic-drone-dataset.zip
import lib.dataset_creator as d
d.train_test_50()
d.validation_split()
#d.data_augment()
#d.data_augment_val()

400
400
300
300


In [0]:
!pip uninstall imgaug && pip install git+https://github.com/aleju/imgaug.git
#model RESNET50 UNET optimizer ADADELTA

epochs = 5

from lib.library_seg.library_seg.keras_segmentation.models.unet import resnet50_unet

n_classes = 23 # Aerial Semantic Segmentation Drone Dataset tree, gras, other vegetation, dirt, gravel, rocks, water, paved area, pool, person, dog, car, bicycle, roof, wall, fence, fence-pole, window, door, obstacle
model3 = resnet50_unet(n_classes, input_height=416, input_width=608, encoder_level=3)

learning_rate = 0.006

from keras.optimizers import Adadelta
optim = Adadelta(learning_rate=learning_rate, rho=0.95)

model3.train( 
    train_images =  "semantic_drone_dataset/original_images/train",
    train_annotations = "semantic_drone_dataset/label_images_semantic/train",
    checkpoints_path = "unet" , epochs=epochs,
    optimizer_name=optim,val_images="semantic_drone_dataset/original_images/val",val_annotations="semantic_drone_dataset/label_images_semantic/val",verify_dataset=False,do_augment=True,augmentation_name="aug_all")

#voglio salvare il modello creato
# serialize model to JSON
model_json = model3.to_json()
with open("resnet50_unet_adagrad.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model3.save_weights("resnet50_unet_adagrad.h5")
print("Saved model to disk")

Uninstalling imgaug-0.4.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/imgaug-0.4.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/imgaug/*
Proceed (y/n)? y
  Successfully uninstalled imgaug-0.4.0
  Cloning https://github.com/aleju/imgaug.git to /tmp/pip-req-build-md1y9vdv
  Running command git clone -q https://github.com/aleju/imgaug.git /tmp/pip-req-build-md1y9vdv
  Created wheel for imgaug: filename=imgaug-0.4.0-cp36-none-any.whl size=965063 sha256=a79cedda18926890468cc53599dd9ce970ccb61a8e82d20668db44a471449166
  Stored in directory: /tmp/pip-ephem-wheel-cache-6fddo84_/wheels/65/3d/94/ee32cbeaa29c473a4db74c2d21904ac747311fdca4732665f0
Successfully built imgaug
ERROR: keras-segmentation 0.3.0 has requirement imgaug==0.2.9, but you'll have imgaug 0.4.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.4.0 which is incompatible.


Using TensorFlow backend.


Epoch 1/5


/usr/local/lib/python3.6/dist-packages/imgaug/imgaug.py:188: DeprecationWarning: Function `SimplexNoiseAlpha()` is deprecated. Use `BlendAlphaSimplexNoise` instead. SimplexNoiseAlpha is deprecated. Use BlendAlphaSimplexNoise instead. The order of parameters is the same. Parameter 'first' was renamed to 'foreground'. Parameter 'second' was renamed to 'background'.
  warn_deprecated(msg, stacklevel=3)
/usr/local/lib/python3.6/dist-packages/imgaug/imgaug.py:188: DeprecationWarning: Function `ContrastNormalization()` is deprecated. Use `imgaug.contrast.LinearContrast` instead.
  warn_deprecated(msg, stacklevel=3)
/usr/local/lib/python3.6/dist-packages/imgaug/imgaug.py:188: DeprecationWarning: Function `FrequencyNoiseAlpha()` is deprecated. Use `BlendAlphaFrequencyNoise` instead. FrequencyNoiseAlpha is deprecated. Use BlendAlphaFrequencyNoise instead. The order of parameters is the same. Parameter 'first' was renamed to 'foreground'. Parameter 'second' was renamed to 'background'.
  warn_de